In [ ]:
import requests
import pandas as pd

In [ ]:
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
%matplotlib inline
#zuerst die nötigen Biblis runterladen.

In [ ]:
r=requests.get("https://kr-geschaefte.zug.ch/gast/geschaefte?commit=Filtern&geschaeft_filter%5Babgeschlossen_bis%5D=&geschaeft_filter%5Babgeschlossen_von%5D=&geschaeft_filter%5Barten_refs%5D%5B%5D=&geschaeft_filter%5Beingereicht_bis%5D=&geschaeft_filter%5Beingereicht_von%5D=&geschaeft_filter%5Bfrist_bis%5D=&geschaeft_filter%5Bhistorische_staende_refs%5D%5B%5D=&geschaeft_filter%5Bkommissionen_refs%5D%5B%5D=&geschaeft_filter%5Bstaende_refs%5D%5B%5D=&geschaeft_filter%5Bstatus_ids%5D%5B%5D=haengig&geschaeft_filter%5Bstatus_ids%5D%5B%5D=abgeschlossen&geschaeft_filter%5Bstatus_ids%5D%5B%5D=&geschaeft_filter%5Btitel%5D=&geschaeft_filter%5Bzustaendig_refs%5D%5B%5D=&page=1&utf8=✓")
soup=BeautifulSoup(r.text, "html.parser")

In [ ]:
soup

In [ ]:
tr_list=soup.find_all("tr") #das ist eine Liste mit allen "tr"-Tags drinn.

In [ ]:
tr_list[3]#durchschauen, ab welcher Position es spannend wird und die Daten sind die wir baruchen.

In [ ]:
tr_liste=tr_list[3:] #da die Daten erst ab der 3. Position verwendet werden sollen, speichere ich die in neue Variable.

In [ ]:
tr_liste[0].find_all("td")[1].text #nun haben wir alle td rausgeholt und sie in einer Liste. 
#dort schauen wir die zweite Position an. Mit .text bringen wir dort den Text alleine raus. 

In [ ]:
td_list=tr_liste[0].find_all("td") #nun eine vorübergehende Variable besteimmen. hier temp

In [ ]:
for td_tag in td_list:
    print(td_tag.text)

In [ ]:
td_list[0]

In [ ]:
#nun müssen wir noch an den Titel rankommen. Der kommt nicht raus, weil er nicht als Text drinn gespeichert ist.
#denn .text sieht nur das was auf der Page ersichtlich ist.
td_list[0].text

In [ ]:
td_list[0].find("a")["href"] #href ist wie ein key. das a müssen wir auslese, da es eine unterkathegorie von td ist.
#alle html-Tags die eine weiter funktion bekanntgeben ist wie ein key 

In [ ]:
#hier würde auch die Geschäftsnummer drinn stehen. Nur ist diese für mich gerade nicht erreichbar.
td_list[0].find("a")

In [ ]:
#nun müssen wir Infos zusammenziehen
#zuerst ein Minidic bauen
#wir ziehen nun ein Element nach dem anderen raus.
td_list = tr_liste[0].find_all("td")
vorlag_nr=td_list[0].text
url = td_list[0].find("a")["href"]
text = td_list[1].text
stand = td_list[2].text
zustaendig = td_list[3].text
geschaeft_art = td_list[4].text
status = td_list[5].text

#weil es relativ übersichtlich ist, haben wir jede Variable einzel definiert und geschrieben. Sonst wärs Loop in loop.
#nun packen wir es in minidict

mini_dict = {"GeschäftsNR": vorlag_nr, "url":url,"Geschäft":text, "Stand":stand, "Zuständigkeit":zustaendig, "Art des Geschäfts": geschaeft_art, "Status":status}


In [ ]:
#jetzt müssen wir dies auf alle "tr"-Positionen (gute_liste) auf der Seite 1 anwenden.
# die tr_liste durch element ersetzen. Bei Element keine "ortsangabe", da es bereits sagt, wo es  ist.
seite_eins = []
for element in tr_liste:
    td_list = element.find_all("td")
    vorlag_nr=td_list[0].text
    url = td_list[0].find("a")["href"]
    text = td_list[1].text
    stand = td_list[2].text
    zustaendig = td_list[3].text
    geschaeft_art = td_list[4].text
    status = td_list[5].text

#weil es relativ übersichtlich ist, haben wir jede Variable einzel definiert und geschrieben. Sonst wärs Loop in loop.
#nun packen wir es in minidict

    mini_dict = {"GeschäftsNR": vorlag_nr, "Geschäft":text, "Stand":stand, "Zuständigkeit":zustaendig, "Art des Geschäfts": geschaeft_art, "Status":status}

    seite_eins.append(mini_dict)


In [ ]:
pd.DataFrame(seite_eins)

In [ ]:
#jetzt machen wir das für alle Seiten. (aber noch nicht ganz automatisiert)
url = "https://kr-geschaefte.zug.ch/gast/geschaefte?commit=Filtern&geschaeft_filter%5Babgeschlossen_bis%5D=&geschaeft_filter%5Babgeschlossen_von%5D=&geschaeft_filter%5Barten_refs%5D%5B%5D=&geschaeft_filter%5Beingereicht_bis%5D=&geschaeft_filter%5Beingereicht_von%5D=&geschaeft_filter%5Bfrist_bis%5D=&geschaeft_filter%5Bhistorische_staende_refs%5D%5B%5D=&geschaeft_filter%5Bkommissionen_refs%5D%5B%5D=&geschaeft_filter%5Bstaende_refs%5D%5B%5D=&geschaeft_filter%5Bstatus_ids%5D%5B%5D=haengig&geschaeft_filter%5Bstatus_ids%5D%5B%5D=abgeschlossen&geschaeft_filter%5Bstatus_ids%5D%5B%5D=&geschaeft_filter%5Btitel%5D=&geschaeft_filter%5Bzustaendig_refs%5D%5B%5D=&page="
#zuerst wollen wir die Seite besuchen. 
#nun müssen wir das machen was wir ganz am Anfang gemacht haben, aber im Forloop.Durch alle 40 Seiten.
# danach müssen wir den anderen Loop ranhängen. Sodass er es für alle 40 Seiten macht.
# wir machen dazu auch eine neue Liste für alle Seiten.
alle_seiten = []
for seite in range(1,41):
    r=requests.get(url+str(seite))
    soup = BeautifulSoup(r.text,"html.parser")
    
    tr_list=soup.find_all("tr")[2:] # 2 weil der Kopf (die ersten zwei Positionen) uns nicht interessiert
    
    for element in tr_list:
        td_list = element.find_all("td")
        vorlag_nr=td_list[0].text
        href = td_list[0].find("a")["href"]
        text = td_list[1].text
        stand = td_list[2].text
        zustaendig = td_list[3].text
        geschaeft_art = td_list[4].text
        status = td_list[5].text
        
        mini_dict = {"GeschäftsNR": vorlag_nr,"url":href,"Geschäft":text, "Stand":stand, "Zuständigkeit":zustaendig, "Art des Geschäfts": geschaeft_art, "Status":status}

        alle_seiten.append(mini_dict)



In [ ]:
df_a=pd.DataFrame(alle_seiten)

In [ ]:
#Hier der Versuch, den "unterloop" als Definition zu machen.
def minidict(listenelement):
 

    for element in listenelement:
        td_list = element.find_all("td")
        href = td_list[0].find("a")["href"]
        text = td_list[1].text
        stand = td_list[2].text
        zustaendig = td_list[3].text
        geschaeft_art = td_list[4].text
        status = td_list[5].text
        
        mini_dict = {"url":href,"Geschäft":text, "Stand":stand, "Zuständigkeit":zustaendig, "Art des Geschäfts": geschaeft_art, "Status":status}

        return mini_dict

In [ ]:
df=pd.DataFrame(alle_seiten)

In [ ]:
df["url"]
#so kann ich einzelne Sachen rausziehen.

In [ ]:
url_liste_ganz=[]
url_anfang_original="https://kr-geschaefte.zug.ch"
for element in df["url"]:
    url_liste_ganz.append(url_anfang_original+element)
    
 

In [ ]:
url_liste_ganz

In [ ]:
#Auf diese URL-s können wir unser System wieder draufschicken.

## Nun gehts in die Unterseiten. 
Dort von diesen erstelle ich ein neues Dataframe mit Geschäftsnummer und Einreichedatum

In [ ]:
rg=requests.get("https://kr-geschaefte.zug.ch/gast/geschaefte/1997")

In [ ]:
soup_g=BeautifulSoup(rg.text, "html.parser")

In [ ]:
soup_g

In [ ]:
tr_g_list=soup_g.find_all("tr")

In [ ]:
tr_g_list

In [ ]:
#hier nochmals die Nummer des Geschäfts
tr_g_list[-1].find_all("td")[0].text

In [ ]:
#Und hier das Einreichedatum
tr_g_list[1].find_all("td")[0].text

In [ ]:
#Nun baue ich zuerst ein Minidict.
gesch_nr = tr_g_list[-1].find_all("td")[0].text
einger_am = tr_g_list[1].find_all("td")[0].text

minidict_g={"GeschäftsNR":gesch_nr, "Einreichedatum":einger_am}


In [ ]:
#For-loop schreiben um Seite für Seite abzurufen und deren Inhalte in das minidict und dieses in eine Liste packen
eingang_list=[]

for geschaeft in url_liste_ganz:
    rg=requests.get(geschaeft)
    soup_g=BeautifulSoup(rg.text, "html.parser")
    tr_g_list=soup_g.find_all("tr")
    gesch_nr = tr_g_list[-1].find_all("td")[0].text
    einger_am = tr_g_list[1].find_all("td")[0].text
      
    
    minidict_g={"GeschäftsNR":gesch_nr, "Einreichedatum":einger_am}
    for key, value in minidict_g.items():
        if value == '':
            minidict_g[key] = 'NaN'  #hier schaue ich noch, dass ich die Leeren Zeilen mit NAN ersetzten kann.
        else:
            minidict_g[key] = value
            
            eingang_list.append(minidict_g)


In [ ]:
df_g=pd.DataFrame(eingang_list)

In [ ]:
#ah, da liegt also das Problem, dass ich nicht plotten kann. es sind biedes Objekte. muss es also umwandeln. hmmm w
pd.set_option("display.max_columns",None)

In [ ]:
df_g

In [ ]:
df_g.dtypes

In [ ]:
#Zwischenstand: nun hat er zwar aus allen leeren Feldern ein NaN gemacht. 
#Trotzdem betrachtet er die Daten noch als Objekte. 

#Ich versuche mal weiter zu machen, indem ich die beiden Dataframes zusammentun
 

In [ ]:
df_mit_dat=pd.merge(df_a, df_g, how="left", on="GeschäftsNR")

In [ ]:
df_mit_dat.dtypes

In [ ]:
#geschafft. nun sind zumindest diese beiden Dataframes vereint. 
# Jetzt muss ich es noch hinkriegen, dass die Objekte zu verwertbaren Zahlen werden.

In [ ]:
#nun der Versuch, die ganze Liste zu sehen.
df_g=pd.DataFrame(eingang_list)
pd.set_option("display.max_rows", 3000)

In [ ]:
#ahhhh, es sind alles Strings. jetzt muss ich die in Daten und Nummern verwandeln
eingang_list

In [ ]:
df_g["GeschäftsNR"]= df_g["GeschäftsNR"].astype(int) #die Vorlagenummer als Zahle

In [ ]:
df_g.dtypes

In [ ]:
df_g["Einreichedatum"] =  pd.to_datetime(df_g["Einreichedatum"], format='%d.%m.%Y') #hier noch das Einreichedatum als Zahl.

In [ ]:
df_g.dtypes

### der Misslungene Versuch die Namens-Seiten die Partei sowie den Namen herauszuziehen.

In [ ]:
liste_parteien=[]
for url_p in df_mit_dat["Namens-url"]:
    rp=requests.get(url_p)
    soup_p=BeautifulSoup(rp.text, "html.parser")
    h1_p_list=soup_p.find_all("h1")
    div_p_list=soup_p.find_all("div")
    partei_p = div_p_list[1].find_all("a")[51].text
    name_p=h1_p_list[0].text
    
    minidict_p={"Name eingereicht":name_p, "Partei":partei_p}
    liste_parteien.append(minidict_p)
    #Das funktinoiert nicht, da die Partei offenbar an verschiedenen Positionen untergebracht ist